In [22]:
from jieba import analyse
import numpy
import gensim

In [ ]:
model_file = 'news_12g_baidubaike_20g_novel_90g_embedding_64.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True)

In [58]:
en_file = "GoogleNews-vectors-negative300.bin"
en_model = gensim.models.KeyedVectors.load_word2vec_format(en_file, binary=True)

In [93]:
# 提取关键词
def keyword_extract(data):
    tfidf = analyse.extract_tags
    keywords = tfidf(data)
    return keywords


# 将文档的每句话进行关键词提取，并将结果保存在txt文件中
def getKeywords(docpath):
    keyword_string_list = list()
    for data in docpath.split("\n"):  # 以换行划分段落，一个data是一段
        keywords_string = '/'.join(keyword_extract(data))
        keyword_string_list.append(keywords_string)
    return keyword_string_list


def word2vec(keywords_string_list, model):
    wordvec_size = 64
    word_vec_all = numpy.zeros(wordvec_size)
    en_wordvec_size = 300
    en_word_vec_all = numpy.zeros(en_wordvec_size)
    keyword_num = 0
    en_keyword_num = 0
    for keywords_string in keywords_string_list:
        keywords_list = keywords_string.split("/")
        for word in keywords_list:
            if word in model:
                keyword_num += 1
                word_vec_all  += model[word]
            elif word in en_model:
                en_keyword_num += 1
                en_word_vec_all += en_model[word]
            else:
                print('word \'{}\' not in word corpus!'.format(word))
    return word_vec_all / keyword_num if keyword_num else word_vec_all, en_word_vec_all / en_keyword_num if en_keyword_num else en_word_vec_all, 1.0 * keyword_num / (keyword_num + en_keyword_num)

# 词向量相似度计算代码：余弦
def simlarityCalu(vector1, vector2, en_vector1, en_vector2, p1_pct, p2_pct):
    print(p1_pct, p2_pct)
    vector1Mod = numpy.sqrt(vector1.dot(vector1)) * p1_pct
    vector2Mod = numpy.sqrt(vector2.dot(vector2)) * p2_pct
    if vector2Mod != 0 and vector1Mod != 0:
        simlarity = (vector1.dot(vector2)) / (vector1Mod * vector2Mod)
    else:
        simlarity = 0
        
    en_vector1Mod = numpy.sqrt(en_vector1.dot(en_vector1)) * (1 - p1_pct)
    en_vector2Mod = numpy.sqrt(en_vector2.dot(en_vector2)) * (1 - p2_pct)
    if en_vector2Mod != 0 and en_vector1Mod != 0:
        en_simlarity = (en_vector1.dot(en_vector2)) / (en_vector1Mod * en_vector2Mod)
    else:
        en_simlarity = 0
    return simlarity + en_simlarity  # 此处只是根据两者百分比求加权算术平均


def get_simlaytyCalu(p1, p2, model):
    p1_keywords = getKeywords(p1)
    p2_keywords = getKeywords(p2)
    p1_vec, en_p1_vec, p1_pct=word2vec(p1_keywords, model) 
    p2_vec, en_p2_vec, p2_pct=word2vec(p2_keywords, model) 
    return p1_keywords, p2_keywords, simlarityCalu(p1_vec,p2_vec, en_p1_vec, en_p2_vec, p1_pct, p2_pct)

In [88]:
p1 = """抽象和封装是互补的概念。一方面，抽象关注对象的行为。另一方面，封装关注对象行为的细节。一般是通过隐藏对象内部状态信息做到封装，
因此，封装可以看成是用来提供抽象的一种策略。"""
 
p2 = """抽象：就是把现实生活中的某一类东西提取出来，用程序代码表示，我们通常叫做类或者接口。抽象包括两个方面：一个是数据抽象，一个是过程抽象。
数据抽象也就是对象的属性。过程抽象是对象的行为特征。
封装：把客观事物封装成抽象的类，并且类可以把自己的数据和方法只让可信的类或者对象操作，对不可信的进行封装隐藏。
封装分为属性的封装和方法的封装。""" 

p1_keywords, p2_keywords, simlarity = get_simlaytyCalu(p1, p2,model)
print(*p1_keywords)
print("\n")
print(*p2_keywords)
print("\n")
print(simlarity)

封装/对象/抽象/行为/关注/互补/隐藏/细节/做到/一方面/另一方面/概念/内部/状态/信息/一般/通过 封装/抽象/看成/用来/策略/提供/一种/因此/可以


抽象/数据抽象/程序代码/接口/现实生活/提取/一类/叫做/一个/通常/或者/东西/过程/出来/方面/包括/两个/表示/就是/我们 数据抽象/对象/抽象/属性/特征/行为/过程/就是 封装/客观事物/可信/抽象/隐藏/对象/操作/不可/并且/或者/方法/数据/进行/自己/可以 封装/属性/分为/方法


0.9100747040392997


In [89]:
# 什么是Java虚拟机？为什么Java被称为平台无关的编程语言

p1 = """A Java virtual machine (JVM) is a process virtual machine that can execute Java bytecode. Each Java source file is compiled into a 
bytecode file, which is executed by the JVM.
Java was designed to allow application programs to be built that could be run on any platform, without having to be rewritten or 
recompiled by the programmer for each separate platform. A Java virtual machine makes this possible, because it is aware of the specific 
instruction lengths and other particularities of the underlying hardware platform."""
 
p2 = """The java source program (. java) is compiled by the compiler into a bytecode file (. class). 
Then the bytecode file will be interpreted by the Java virtual machine as machine code (machine code for different platforms is
different). Using machine code to operate hardware and operating system, because different platforms have different JVMs, 
they can interpret the same. class file into machine code needed by different platforms. It is precisely because of the existence of 
JVM that Java is called a platform-independent programming language.""" 

p1_keywords, p2_keywords, simlarity = get_simlaytyCalu(p1, p2,model)
print(*p1_keywords)
print("\n")
print(*p2_keywords)
print("\n")
print(simlarity)

Java/virtual/machine/JVM/process/execute/bytecode/Each/source/file/compiled/into bytecode/file/executed/JVM Java/was/designed/allow/application/programs/built/could/run/any/platform/without/having/rewritten recompiled/programmer/each/separate/platform/Java/virtual/machine/makes/possible/because/aware/specific instruction/lengths/other/particularities/underlying/hardware/platform


java/source/program/compiled/compiler/into/bytecode/file/class machine/code/bytecode/file/will/interpreted/Java/virtual/different/platforms different/Using/machine/code/operate/hardware/operating/system/because/platforms/JVMs they/interpret/same/class/file/into/machine/code/needed/different/platforms/precisely/because/existence JVM/Java/called/platform/independent/programming/language


0.9321615153564592


In [94]:
p1 = """把一组数据结构和处理它们的方法组成对象（object），把相同行为的对象归纳为类（class），通过类的封装（encapsulation）隐藏内部细节，通过继承（inheritance）实现类的泛化（generation）/特化（specialization），通过多态（polymorphism）实现基于对象类型的动态分派（dynamic dispatch）"""
 
p2 = """面向对象是一种思想，是基于面向过程而言的，就是说面向对象是将功能等通过对象来实现，将功能封装进对象之中，
让对象去实现具体的细节；这种思想是将数据作为第一位，而方法或者说是算法作为其次，
这是对数据一种优化，操作起来更加的方便，简化了过程。面向对象有三大特征：封装性、继承性、多态性， 
其中封装性指的是隐藏了对象的属性和实现细节，仅对外提供公共的访问方式，这样就隔离了具体的变化，便于使用，
提高了复用性和安全性。对于继承性，就是两种事物间存在着一定的所属关系，那么继承的类就可以从被继承的类中获得一些属性和方法；
这就提高了代码的复用性。继承是作为多态的前提的。多态是说父类或接口的引用指向了子类对象，这就提高了程序的扩展性，
也就是说只要实现或继承了同一个接口或类，那么就可以使用父类中相应的方法，提高程序扩展性，但是多态有一点不好之处在于：
父类引用不能访问子类中的成员的特有方法和属性。
！""" 

p1_keywords, p2_keywords, simlarity = get_simlaytyCalu(p1, p2,model)
print(*p1_keywords)
print("\n")
print(*p2_keywords)
print("\n")
print(simlarity)

word '有三大' not in word corpus!
word '' not in word corpus!
0.55 1.0
对象/泛化/多态/通过/object/为类/class/encapsulation/inheritance/generation/specialization/polymorphism/dynamic/dispatch/数据结构/封装/特化/实现/分派/归纳


面向对象/对象/功能/封装/就是说/面向/基于/而言/之中/思想/实现/过程/一种/通过 算法/作为/第一位/或者说/细节/其次/对象/思想/具体/方法/数据/实现/这种 多态性/有三大/继承性/面向对象/封装/简化/优化/方便/特征/操作/更加/数据/过程/一种/起来 性指/封装/隔离/隐藏/属性/访问/细节/便于/公共/对外/对象/具体/方式/变化/提供/实现/使用/其中/这样 继承/继承性/复用/安全性/属性/所属/事物/两种/方法/获得/关系/提高/那么/存在/对于/一定/一些/就是/可以 多态/子类/扩展性/父类/复用/提高/接口/指向/代码/引用/继承/前提/程序/对象/作为 扩展性/多态/或类/父类/接口/同一个/也就是说/继承/程序/相应/在于/不好/只要/方法/一点/提高/实现/那么/使用/但是 子类/父类/属性/访问/引用/特有/成员/方法/不能 


1.4508565172982744
